In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient("localhost", 27017)
db = client.axis_project

In [ ]:
#read the data from mongodb collection 'cochin1 and store it in a dataframe
input_data = db.cochin
data = pd.DataFrame(list(input_data.find()))
del data['_id']
data

In [4]:
data.to_excel('C:/Users/Praveer/Desktop/Correlation_of_Weather_Data_and_Hotel_Occupancy/Hotel_Cochin_Analysis/Results/Hotel_Data_from_mongo.xlsx')

In [ ]:
#finding the Stay_date for each day (unwinding the data) (from 2015-01-01 to 2016-12-31 )

pipeline = [ { "$unwind" : "$Stay_date" } ] 
data_for_stay_date = pd.DataFrame(list(db.cochin.aggregate(pipeline)))
data_for_stay_date = data_for_stay_date[(data_for_stay_date.Stay_date >= '2015-01-01') & (data_for_stay_date.Stay_date <= '2016-12-31')]
del data_for_stay_date['_id']
data_for_stay_date

In [6]:
data_for_stay_date.to_excel('C:/Users/Praveer/Desktop/Correlation_of_Weather_Data_and_Hotel_Occupancy/Hotel_Cochin_Analysis/Results/Stay_date.xlsx')

In [ ]:
#finding the room counts from 2015-01-01 to 2016-12-31

data_count = pd.DataFrame(list
                              ( db.cochin.aggregate( [ 
                                                        { "$unwind" : "$Stay_date" },
                                                        {"$group" :
                                                            { "_id":"$Stay_date",
                                                              "counts_of room":{"$sum":1}}}
                                                    ])
                              )
                         )
data_count = data_count[(data_count._id >= '2015-01-01') & (data_count._id <= '2016-12-31')]
data_count

In [ ]:
#Average room rates from 2015-01-01 to 2016-12-31

data_avg = pd.DataFrame(list
                              ( db.cochin.aggregate( [ 
                                                        { "$unwind" : "$Stay_date" },
                                                        { "$group" :
                                                            { "_id":"$Stay_date",
                                                              "counts_of room":{"$sum":1},
                                                              "Average" : { "$avg" : "$Room_rate"}
                                                            }
                                                        }
                                                    ])
                              )
                         )

data_avg = data_avg[(data_avg._id >= '2015-01-01') & (data_avg._id <= '2016-12-31')]
data_avg.set_index('_id')
data_avg

In [ ]:
#Total Average room rates from 2015-01-01 to 2016-12-31
data_tot_avg = data_avg["Average"].mean()
data_tot_avg

In [ ]:
#Total Average room rates
data_total_avg = pd.DataFrame(list
                            (db.cochin.aggregate( [ 
                                                    { "$unwind" : "$Stay_date" },
                                                    { "$group" :
                                                                 { "_id":"$ Room_rate",
                                                                   "counts_of room":{"$sum":1},
                                                                   "Average" : { "$avg" : "$Room_rate"}
                                                                 }
                                                     },
                                                     { "$group" :
                                                                 { "_id":"null",
                                                                   "Average" : { "$avg" : "$Average"}
                                                                 }
                                                     }
                                                    ])
                             )
                       )
data_total_avg

In [ ]:
#Finding pick up dates
data_pick_up = pd.DataFrame(list
                                (db.cochin.aggregate( [
                                                        {"$project" : { "booking_date" : { "$substr": [ "$Booking_date", 0, 10 ] }
            
                                                                        } },
      
                                                        { "$group" : { "_id": "$booking_date", 
                                                                       "count" : {"$sum":1} 
                                                                     }
                                                        } 
                                                       ]
                                                        )))
data_pick_up = data_pick_up[(data_pick_up._id >= '2015-01-01') & (data_pick_up._id <= '2016-12-31')]
data_pick_up = data_pick_up.sort_values(by=['_id'], ascending=[True])
data_pick_up